In [ ]:
# Goal: see if SV3 10-pass region with PIP clustering is matched by various things from my fiber incompleteness handled version of the data

# Run sv3-tests.sh

In [ ]:
import sys, os
import numpy as np
from astropy.table import Table, vstack
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, project_to_multipoles, project_to_wp, utils, setup_logging
from dataloc import *
from desiclusteringtools import *
import matplotlib.pyplot as plt

In [ ]:
fn_pattern = f'BGS_BRIGHT_0_full.ran.fits'
dirin = f"/global/cfs/cdirs/desi/users/ianw89/newclustering/SV3/LSS/fuji/LSScats/3.1/"
tbl = Table.read(os.path.join(dirin, fn_pattern))

# Convert RA, DEC to radians for Mollweide projection
ra = np.array(tbl['RA'])
dec = np.array(tbl['DEC'])
ra_rad = np.radians(ra)
ra_rad = np.remainder(ra_rad + 2*np.pi, 2*np.pi)  # ensure [0, 2pi]
ra_rad[ra_rad > np.pi] -= 2*np.pi  # shift to [-pi, pi]
dec_rad = np.radians(dec)
ra_rad = ra_rad + np.pi
ra_rad[ra_rad > np.pi] -= 2*np.pi

plt.figure(figsize=(10, 5))
ax = plt.subplot(111, projection='mollweide')
ax.hexbin(ra_rad, dec_rad, gridsize=300, cmap='viridis', mincnt=1)
ax.set_xlabel('RA')
ax.set_ylabel('DEC')
ax.set_title('Mollweide Projection Randoms')
ax.grid(True)
plt.show()

In [ ]:
fn_pattern = f'BGS_BRIGHT_10p_clustering.dat.fits'
dirin = f"/global/cfs/cdirs/desi/users/ianw89/newclustering/SV3/LSS/fuji/LSScats/3.1/ALL"
tbl = Table.read(os.path.join(dirin, fn_pattern))
print(f"Columns in the table: {tbl.colnames}")
print(f"Rows in the table: {len(tbl)}")
# TODO Code to make a GroupCatalog object become something like this table. Use functions in mkCat_subSamp maybe?

In [ ]:
survey = 'SV3'
verspec = 'fuji'
ver = '3.1'
color = 'ALL'
dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER_NEW, survey, 'LSS', verspec, 'LSScats', ver, color, 'rppi')
#wp_fn = os.path.join(dir, f'wp_BGS_BRIGHT_10p_NGC_0.001_0.5_pip_bitwise_custom1_njack0_nran1_split20.txt')
allcounts_fn = os.path.join(dir, f'allcounts_BGS_BRIGHT_10p_NGC_0.001_0.5_pip_bitwise_custom_njack16_nran18_split20.npy')
tpc = TwoPointEstimator.load(allcounts_fn)
rp, wp, cov = tpc.get_corr(return_sep=True, return_cov=True, mode='wp')

color = 'SF'
dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER_NEW, survey, 'LSS', verspec, 'LSScats', ver, color, 'rppi')
#wp_fn = os.path.join(dir, f'wp_BGS_BRIGHT_10p_NGC_0.001_0.5_pip_bitwise_custom1_njack0_nran1_split20.txt')
allcounts_fn = os.path.join(dir, f'allcounts_BGS_BRIGHT_SF_10p_NGC_0.001_0.5_pip_bitwise_custom_njack16_nran18_split20.npy')
tpc = TwoPointEstimator.load(allcounts_fn)
rp2, wp2, cov2 = tpc.get_corr(return_sep=True, return_cov=True, mode='wp')

color = 'Q'
dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER_NEW, survey, 'LSS', verspec, 'LSScats', ver, color, 'rppi')
#wp_fn = os.path.join(dir, f'wp_BGS_BRIGHT_10p_NGC_0.001_0.5_pip_bitwise_custom1_njack0_nran1_split20.txt')
allcounts_fn = os.path.join(dir, f'allcounts_BGS_BRIGHT_Q_10p_NGC_0.001_0.5_pip_bitwise_custom_njack16_nran18_split20.npy')
tpc = TwoPointEstimator.load(allcounts_fn)
rp3, wp3, cov3 = tpc.get_corr(return_sep=True, return_cov=True, mode='wp')

In [ ]:
plt.figure(figsize=(5, 5))
plt.xscale('log')
plt.yscale('log')
plt.ylim(2, 4000)
plt.ylabel(r'$w_p(r_p)$')
plt.xlabel(r'$r_p$ [Mpc/h]')

plt.errorbar(rp, wp, label='Orig All PIP', yerr=np.sqrt(cov.diagonal()), color='k')
plt.errorbar(rp2, wp2, label='Orig SF PIP', yerr=np.sqrt(cov2.diagonal()), color='b')
plt.errorbar(rp3, wp3, label='Orig Q PIP', yerr=np.sqrt(cov3.diagonal()), color='r')
plt.legend()

# The hope is that all my group catalogs with fiber incompleteness handled properly will match these PIP results